In [ ]:
# необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
import copy

from matplotlib import cm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [ ]:
# Функции для создания выборки данных

def make_DATA(low, high, size, D_size): # Создание выборки и запись в файл
  #X = [np.random.normal(low, high, size) for i in range(D_size)] # нормальный закон распределения с матожиданием low и дисперсией high
  X = [np.random.uniform(low, high, size) for i in range(D_size)] # равномерный закон распределения на отрезке [low, high]

  df = pd.DataFrame(X)
  df.to_csv('DATA.csv', index=False) # создание файла с данными
  return df

def func(x, ab, c): # Создание отклика системы из неизвестной функции
  ans=[]
  match flag: # выбор неизвестной функции
    case 1: # ф. Ишигами
      ans = [math.sin(x['0'][i]) + ab[0] * ((math.sin(x['1'][i]))**2) + ab[1] * (0**4) * math.sin(x['0'][i]) for i in range(len(x))]
    case 2:  # простая функция
      ans = [(x['0'][i] - 0.5)**2 for i in range(len(x))]
    case 4:  # Соболь
      for i in range(len(x)):
        ans.append(1)
        for k in range(len(c)):
          ans[i]*=(abs(4*x[str(k)][i]-2)+c[k])/(1+c[k])
  return ans

def eps_f(beta, D_size): # Создание шума
  eps = [np.random.normal(0, 1 / beta**0.5) for i in range(D_size)] # нормальный закон распределения с матожиданием 0 и дисперсией 1 / beta

  df = pd.DataFrame(eps)
  df.to_csv('eps.csv', index=False) # создание файла с данными
  return df

In [ ]:
flag = 2 # выбор неизвестной функции

# 1 - функция Ишигами
# 2 - гладкая функция
# 3 - не моделированные данные
# 4 - функция Соболя

In [ ]:
low = 0 # левая граница отрезка или матожидание
high = 1 # правая граница отрезка или дисперсия
if flag == 4:
  low = -math.pi # левая граница отрезка
  high = math.pi # правая граница отрезка
c = [-0.5, 0.5] # параметр для ф. Соболя
match flag: # количество признаков
  case 1:
    size = 2
  case 2:
    size = 1
  case 4:
    size = len(c)
D_size = 1000 # размер выборки
ab = [7,0.1] # параметр для ф. Ишигами
beta_eps = 5 # истинный гиперпараметр для шума
beta = 10 # гиперпараметр, выбранный случайно
r = 5 # максимальная степень полинома К-Г
test_size = 0.2

In [ ]:
if flag == 3:
  name= ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

  data = pd.read_csv('housing.csv',delim_whitespace=True,names=name)
  data = data.drop(['CHAS','RAD','ZN'],axis =1)

  columns_to_normalize = ['CRIM', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'LSTAT', 'MEDV','B']

  min_values = data[columns_to_normalize].min()
  max_values = data[columns_to_normalize].max()

  data[columns_to_normalize] = (data[columns_to_normalize])/(max_values - min_values) * math.pi # нормализация

  x = data.drop('MEDV', axis = 1)

  x = x.rename(columns={'CRIM':'0', 'INDUS':'1', 'NOX':'2', 'RM':'3', 'AGE':'4', 'DIS':'5',  'TAX':'6', 'PTRATIO':'7', 'LSTAT':'8','B':'9'})
  x = x.reset_index(drop= True)

else:
  #x = make_DATA(low, high, size, D_size) # генерируем данные в файл
  x = pd.read_csv('DATA.csv') # считываем данные из файла

x

,0
0,0.803574
1,0.766581
2,0.327652
3,0.607529
4,0.489521
...,...
995,0.654341
996,0.662915
997,0.397215
998,0.687462


In [ ]:
if flag == 3:
  y = data['MEDV'] # сохранение выходных данных
  y = y.reset_index(drop= True)
  df = pd.DataFrame({'y':y})
else:
  f = func(x, ab, c) # вычисляем истинные значения отклика
  #eps_f = eps_f(beta_eps, D_size) # генерируем шум в файл
  eps_f = pd.read_csv('eps.csv') # считываем шум из файла
  y = [f[i] + eps_f['0'][i] for i in range(D_size)] # получаем зашумленный отклик модели

  # Выводим полученные результаты вычислений в виде таблицы
  df = pd.DataFrame({'func':f, 'eps':eps_f['0'], 'y':y})
df

,func,eps,y
0,0.092157,-0.914386,-0.822229
1,0.071065,0.613849,0.684914
2,0.029704,-0.966238,-0.936534
3,0.011563,0.168639,0.180201
4,0.000110,-0.111843,-0.111733
...,...,...,...
995,0.023821,1.052573,1.076394
996,0.026541,0.319785,0.346326
997,0.010565,-0.020816,-0.010251
998,0.035142,-0.685251,-0.650109


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42) # деление на обучающую и тестовую выборку
x = x_train.reset_index(drop= True)
x_test = x_test.reset_index(drop= True)
if type(y) == list:
  y = y_train
  y_test = y_test
else:
  y = y_train.tolist()
  y_test = y_test.tolist()

D_size = D_size * (1 - test_size) # новый размер выборки

In [ ]:
def make_KG(x, r): # полиномы Колмагорова-Габора
  X = []
  # Начинаем строку в таблице Х с 1
  for i in range(len(x)):
    X.append([])
    X[i].append(1)

    # Далее составляем всевозможные сочетания признаков
    if r >= 1:
      for j in range(x.shape[1]):
          X[i].append(x[str(j)][i]) # например: х_1, х_2

    if r >= 2:
      for j in range(x.shape[1]):
          for k in range(j, x.shape[1]):
              X[i].append(x[str(j)][i]*x[str(k)][i]) # например: х_1^2, x_1 * x_2, x_2^2

    if r >= 3:
      for j in range(x.shape[1]):
        for k in range(j, x.shape[1]):
          for l in range(k, x.shape[1]):
            X[i].append(x[str(j)][i]*x[str(k)][i]*x[str(l)][i])

    if r >= 4:
      for j in range(x.shape[1]):
        for k in range(j, x.shape[1]):
          for l in range(k, x.shape[1]):
            for q in range(l, x.shape[1]):
              X[i].append(x[str(j)][i]*x[str(k)][i]*x[str(l)][i]*x[str(q)][i])

    if r >= 5:
      for j in range(x.shape[1]):
        for k in range(j, x.shape[1]):
          for l in range(k, x.shape[1]):
            for q in range(l, x.shape[1]):
              for e in range(q, x.shape[1]):
                X[i].append(x[str(j)][i]*x[str(k)][i]*x[str(l)][i]*x[str(q)][i]*x[str(e)][i])

  return X

In [ ]:
def make_Furie(x, r): # полиномы Фурье
  X = []
  # Начинаем строку в таблице Х с 1
  for i in range(len(x)):
    X.append([])
    X[i].append(1)

    if r >= 1:
      for j in range(x.shape[1]):
          X[i].append(math.cos(x[str(j)][i])) # например: cos(x)
          X[i].append(math.sin(x[str(j)][i]))

    if r >= 2:
      for j in range(x.shape[1]):
          X[i].append(math.cos(2*x[str(j)][i])) # например: cos(2x)
          X[i].append(math.sin(2*x[str(j)][i]))

    if r >= 3:
      for j in range(x.shape[1]):
          X[i].append(math.cos(3*x[str(j)][i])) # например: cos(3x)
          X[i].append(math.sin(3*x[str(j)][i]))

    if r >= 4:
      for j in range(x.shape[1]):
          X[i].append(math.cos(4*x[str(j)][i])) # например: cos(4x)
          X[i].append(math.sin(4*x[str(j)][i]))

    if r >= 5:
      for j in range(x.shape[1]):
          X[i].append(math.cos(5*x[str(j)][i])) # например: cos(5x)
          X[i].append(math.sin(5*x[str(j)][i]))

  return X

In [ ]:
# матрицы порожденных признаков
if flag == 4 or flag == 3: X = [pd.DataFrame(make_Furie(x,i)) for i in range(1, r+1)]
else: X = [pd.DataFrame(make_KG(x,i)) for i in range(1, r+1)]

In [ ]:
def make_w0(beta, X, A): # Метод гребневой регрессии
  M = beta * np.dot(X.T, X) + np.linalg.inv(A)
  return np.dot(np.dot(np.linalg.inv(M), X.T) * beta, y)

In [ ]:
def make_LSM(X): # Метод МНК
  M = np.dot(X.T, X)
  return(np.dot(np.dot(np.linalg.inv(M), X.T), y))

In [ ]:
w=[]
for i in range(r): w.append(make_w0(beta, X[i], np.eye(X[i].shape[1])))

print('w = ')
pd.DataFrame(w)

w = 


,0,1,2,3,4,5
0,0.059114,0.054134,NaN,NaN,NaN,NaN
1,0.177664,-0.641225,0.694141,NaN,NaN,NaN
2,0.186629,-0.750207,0.971467,-0.188505,NaN,NaN
3,0.198495,-0.815949,0.803527,0.557138,-0.550041,NaN
4,0.199876,-0.778208,0.566144,0.649550,0.087323,-0.552817


In [ ]:
w_LSM = []
for i in range(r): w_LSM.append(make_LSM(X[i]))
pd.DataFrame(w_LSM)

,0,1,2,3,4,5
0,0.059102,0.054171,NaN,NaN,NaN,NaN
1,0.184232,-0.677134,0.728778,NaN,NaN,NaN
2,0.242064,-1.317621,2.274015,-1.014380,NaN,NaN
3,0.195093,-0.480854,-1.344580,4.497776,-2.722453,NaN
4,0.149159,0.730161,-9.508921,25.888276,-26.559936,9.4855


In [ ]:
g=[] # найденные значения методом гребневой регрессии
for i in range(r):
  g.append(np.dot(X[i], w[i]))
print(pd.DataFrame({'g[0]':g[0],'g[1]':g[1],'g[2]':g[2],'g[3]':g[3],'g[4]':g[4],'y':y}))
#print(pd.DataFrame({'g[0]':g[0],'g[1]':g[1],'g[2]':g[2],'g[3]':g[3],'g[4]':g[4],'f':f}))

         g[0]      g[1]      g[2]      g[3]      g[4]         y
0    0.091278  0.041722  0.044298  0.045667  0.043551 -0.402043
1    0.097995  0.075192  0.079102  0.087011  0.091231 -0.191405
2    0.099077  0.082588  0.086398  0.094831  0.099974 -0.270315
3    0.101402  0.100345  0.103550  0.112198  0.118787 -0.126039
4    0.102518  0.109777  0.112481  0.120691  0.127586 -0.240780
..        ...       ...       ...       ...       ...       ...
795  0.083213  0.029771  0.028551  0.022049  0.016707 -0.022177
796  0.077675  0.039408  0.036011  0.028047  0.024375  0.420815
797  0.068622  0.086454  0.083812  0.082468  0.084169  0.699155
798  0.072653  0.060713  0.056819  0.051252  0.050621 -0.349801
799  0.109210  0.178722  0.174941  0.169677  0.168176  0.511010

[800 rows x 6 columns]


In [ ]:
er = []
for i in range(r):
  e = np.square(np.subtract(g[i], y)).mean()
  er.append(e)
  print(f"Ошибка при r = {i+1} {e}")

Ошибка при r = 1 0.19758309553247153
Ошибка при r = 2 0.19496691770983893
Ошибка при r = 3 0.19485776538774047
Ошибка при r = 4 0.19464765176928325
Ошибка при r = 5 0.19456004020450035


In [ ]:
def errors(X_new, w_new_B, X_old_B, w_old_B): # нахождение значений ошибок

  #if flag == 3:
  if True:
    # ошибки после первого этапа БП
    error_MAE_new = mean_absolute_error(np.dot(X_new, w_new_B), y)
    error_MSE_new = np.square(np.subtract(np.dot(X_new, w_new_B),y)).mean()

    # ошибки после предыдущего первого этапа БП
    error_MAE_old = mean_absolute_error(np.dot(X_old_B, w_old_B), y)
    error_MSE_old = np.square(np.subtract(np.dot(X_old_B, w_old_B),y)).mean()

  return error_MAE_new, error_MSE_new, error_MAE_old, error_MSE_old

In [ ]:
def make_param(w, X, beta): # малый цикл

  dif = 1 # разница между старой и новой beta
  p_iter = 1 # номер итерации

  while dif > 0.01:

    Hd = np.dot(X.T, X) * beta # гессиан, не зависящий от А

    lmb, eigenvectors = np.linalg.eig(Hd) # собственные числа гессиана

    alpha = [] # диагональные элементы матрицы A = I[1 / alpha]
    for i in range(len(lmb)):
      alpha.append((-lmb[i] + (lmb[i]**2 + 4 * lmb[i] / (w[i]**2))**0.5) / 2)

    gamma = sum([lmb[i] / (lmb[i] + alpha[i]) for i in range(len(lmb))])

    g = np.dot(X, w)
    Ed = sum([(g[i] - y[i])**2 for i in range(len(y))]) / 2 # функция регрессионных невязок

    beta_new = (D_size - gamma) / (2 * Ed) # пересчет beta

    dif = abs(beta_new - beta) # пересчет разницы значений бет
    beta = beta_new # заменяем бету на новую

    p_iter+=1

  return beta, alpha

In [ ]:
def first_step(w, X, beta): # первый этап - большой цикл поиска w

  dif = 1 # разница между значениями векторов весов
  Big_iter = 1 # номер итерации

  w_old = w
  beta_w = beta

  while dif > 0.01:
    beta_w, alpha_w = make_param(copy.deepcopy(w), copy.deepcopy(X), copy.deepcopy(beta_w)) # находим новые гиперпараметры
    A = np.eye(len(alpha_w)) * [1 / alpha_w[i] for i in range(len(alpha_w))] # составляем ковариационную матрицу
    w = make_w0(copy.deepcopy(beta_w), copy.deepcopy(X), copy.deepcopy(A)) # заново пересчитываем вектор весов с новыми гиперпараметрами

    Big_iter += 1

    dif=sum([abs(w_old[i]-w[i]) for i in range(len(w))])
    w_old = w # заменяем вектор весов на новый

  return w_old, beta_w, alpha_w

In [ ]:
def second_step(alpha_w, w_new_B, X_new, k, iter): # второй этап

  print(f'индекс = {len(alpha_w) + k}')
  X_new = X_new.drop(iter, axis = 1) # убираем столбец с наименьшим значением гиперпараметра
  X_new.insert(iter, iter, X[l + 1][len(alpha_w) + k]) # добовляем столбец с новой базисной функцией

  w_new = w_new_B
  w_new[iter] = w[l + 1][iter] # то же самое, что приметить метод гребневой регресии
                               # как бы заново ищем w0

  return X_new, w_new

In [ ]:
def Bayes(w, X, beta): # двухуровневый метод байеса

  # перебираем базисные функции из Х с большей максимальной степенью полинома
  w_new = w[l] # аналог w0
  X_new = X[l]
  beta_w = copy.deepcopy(beta)
  w_old_B = w[l]
  X_old_B = X[l]
  beta_old = copy.deepcopy(beta)
  alpha_old = np.ones(X[l].shape[1])
  count = np.arange(0, X[l+1].shape[1] - X[l].shape[1]).tolist() # список новых базисных функций
  count_safe = np.arange(X[l].shape[1]).tolist()
  extra = -1 # баз. ф., которую в будущем надо исключить из рассмотрения

  # Для начала выполним только первый этап

  print("___________________________ПЕРВЫЙ ЭТАП___________________________\n")

  w_new_B, beta_w, alpha_w = first_step(copy.deepcopy(w_new), copy.deepcopy(X_new), copy.deepcopy(beta_w)) # результат перого этапа БП
  error_MAE_new, error_MSE_new, error_MAE_old, error_MSE_old = errors(X_new, w_new_B, X_old_B, w_old_B) # считаем ошибки

  w_first = copy.copy(w_new_B) # сохраняем значение после первого этапа ( без вторых )

  # выводим результаты первого этапа
  w_old_B = copy.deepcopy(w_new_B)
  X_old_B = copy.deepcopy(X_new)
  beta_old = copy.deepcopy(beta_w)
  alpha_old = copy.deepcopy(alpha_w)

  print(f'w = {w_old_B}')
  print(f"beta = {beta_old}\n")

  # теперь попробуем еще уменьшить ошибку за счет добавления второго этапа

  for q in range(X[l+1].shape[1] - X[l].shape[1]):

    print("___________________________ВТОРОЙ ЭТАП___________________________\n")

    print(f"alpha = {alpha_old}")

    min = 10**30
    iter = -1
    for i in range(len(alpha_old)): # ищем индекс min. значения гиперпараметра
      if min > alpha_old[i]:
        min = alpha_old[i]
        iter = i

    print(f'заменяем элемент с индексом {iter}\n')

    alpha_old_const = copy.deepcopy(alpha_old)
    w_old_B_const = copy.deepcopy(w_old_B)
    X_old_B_const = copy.deepcopy(X_old_B)


    for k in count: # находим лучшую базисную функцию
      X_new, w_new = second_step(alpha_old_const, w_old_B_const, X_old_B_const, k, iter) # убираем элемент на месте iter и добавляем базисную функцию на месте k

      # проверяем, уменьшилась ли она после первого этапа с новой баз. ф.
      w_new_B_for, beta_w_for, alpha_w_for = first_step(copy.deepcopy(w_new), copy.deepcopy(X_new), copy.deepcopy(beta_old)) # результат перого этапа БП
      error_MAE_new, error_MSE_new, error_MAE_old, error_MSE_old = errors(X_new, w_new_B_for, X_old_B, w_old_B) # считаем ошибки

      if error_MAE_new <= error_MAE_old and error_MSE_new <= error_MSE_old:
        print(f"\n _____Удалось найти более подходящуюю базисную функцию (ее номер {len(alpha_w) + k})_____\n")

        w_old_B = copy.deepcopy(w_new_B_for)
        X_old_B = copy.deepcopy(X_new)
        beta_old = copy.deepcopy(beta_w_for)
        alpha_old = copy.deepcopy(alpha_w_for)
        extra = k
        print(f'w = {w_old_B}')
        print(f"beta = {beta_old}")

    if extra in count:
      print(f"\n _____Удаляем из списка новых баз. ф. использованную функцию под номером {len(alpha_old) + extra} _____\n")
      count.remove(extra) # удаляем из списка новых баз. ф. использованную
      count_safe.remove(iter)
      count_safe.insert(iter, len(alpha_old) + extra)
    else:
      print(f"\n _____НЕ удалось найти более подходящую базисную функцию _____\n")
      break # если значение extra не изменилось, то не был найден лучший вариант, следовательно надо выходить из цикла

  print(f"\n Результат: w = {w_old_B}")
  print(f"\n            beta = {beta_old}")
  print(f'\n            base functions = {count_safe}')

  return w_old_B, X_old_B, beta_old, w_first, count_safe

In [ ]:
l = 3  -1 # выбираем максимальную степень полинома на первом этапе

In [ ]:
# ищим значения

X_old = X[l]
X_LSM = X[l]
w0 = w[l]

w[l], X[l], beta_new, w_first, count_safe = Bayes(copy.deepcopy(w), copy.deepcopy(X), copy.deepcopy(beta))

if flag == 4 or flag == 3: Xx = pd.DataFrame(make_Furie(x_test,l+2))
else: Xx = pd.DataFrame(make_KG(x_test,l+2))
XX = []
for j in range(len(count_safe)):
  XX.insert(j, Xx[count_safe[j]])
XX = pd.DataFrame(np.array(XX).T, columns = count_safe)

if flag == 4 or flag == 3: X_old_ = pd.DataFrame(make_Furie(x_test,l+1))
else: X_old_ = pd.DataFrame(make_KG(x_test,l+1))
X_LSM_ = X_old_


print('Ошибка на обучающей выборке')
print('MAE')
print(f'w2_{l + 1}: {mean_absolute_error(np.dot(X[l], w[l]), y)} второй этап')
print(f'w1_{l + 1}: {mean_absolute_error(np.dot(X_old, w_first), y)} первый этап')
print(f'w0_{l + 1}: {mean_absolute_error(np.dot(X_old, w0), y)} гребневая регрессия')
print(f'w_LSM_{l + 1}: {mean_absolute_error(np.dot(X_LSM, w_LSM[l]), y)} МНК')
print('MSE')
print(f'w2_{l + 1}: {np.square(np.subtract(np.dot(X[l], w[l]),y)).mean()} второй этап')
print(f'w1_{l + 1}: {np.square(np.subtract(np.dot(X_old, w_first),y)).mean()} первый этап')
print(f'w0_{l + 1}: {np.square(np.subtract(np.dot(X_old, w0),y)).mean()} гребневая регрессия')
print(f'w_LSM_{l + 1}: {np.square(np.subtract(np.dot(X_LSM, w_LSM[l]),y)).mean()} МНК')
print('\n')
print('Ошибка на тестовой выборке')
print('MAE')
print(f'w2_{l + 1}: {mean_absolute_error(np.dot(XX, w[l]), y_test)} второй этап')
print(f'w1_{l + 1}: {mean_absolute_error(np.dot(X_old_, w_first), y_test)} первый этап')
print(f'w0_{l + 1}: {mean_absolute_error(np.dot(X_old_, w0), y_test)} гребневая регрессия')
print(f'w_LSM_{l + 1}: {mean_absolute_error(np.dot(X_LSM_, w_LSM[l]), y_test)} МНК')
print('MSE')
print(f'w2_{l + 1}: {np.square(np.subtract(np.dot(XX, w[l]),y_test)).mean()} второй этап')
print(f'w1_{l + 1}: {np.square(np.subtract(np.dot(X_old_, w_first),y_test)).mean()} первый этап')
print(f'w0_{l + 1}: {np.square(np.subtract(np.dot(X_old_, w0),y_test)).mean()} гребневая регрессия')
print(f'w_LSM_{l + 1}: {np.square(np.subtract(np.dot(X_LSM_, w_LSM[l]),y_test)).mean()} МНК')

___________________________ПЕРВЫЙ ЭТАП___________________________

w = [ 9.72793745e-02 -1.65855504e-01 -1.35687327e-08  2.81660353e-01]
beta = 5.096754854489672

___________________________ВТОРОЙ ЭТАП___________________________

alpha = [101.60193625447255, 33.0046606188601, 85232555.74486054, 1.9040508456877867]
заменяем элемент с индексом 3

индекс = 4

 _____НЕ удалось найти более подходящую базисную функцию _____


 Результат: w = [ 9.72793745e-02 -1.65855504e-01 -1.35687327e-08  2.81660353e-01]

            beta = 5.096754854489672

            base functions = [0, 1, 2, 3]
Ошибка на обучающей выборке
MAE
w2_3: 0.3550736073146694 второй этап
w1_3: 0.3550736073146694 первый этап
w0_3: 0.3545584401766788 гребневая регрессия
w_LSM_3: 0.3545577403713704 МНК
MSE
w2_3: 0.19570740410797313 второй этап
w1_3: 0.19570740410797313 первый этап
w0_3: 0.19485776538774047 гребневая регрессия
w_LSM_3: 0.1946256294736066 МНК


Ошибка на тестовой выборке
MAE
w2_3: 0.3739450473355177 второй этап
w1

In [ ]:
pd.DataFrame({'w_LSM':w_LSM[l],'w0':w0,'w1':w_first,'w2':w[l]})

,w_LSM,w0,w1,w2
0,0.242064,0.186629,9.727937e-02,9.727937e-02
1,-1.317621,-0.750207,-1.658555e-01,-1.658555e-01
2,2.274015,0.971467,-1.356873e-08,-1.356873e-08
3,-1.014380,-0.188505,2.816604e-01,2.816604e-01


In [ ]:
beta_new # найденный гиперпараметр шума

5.096754854489672

In [ ]:
s = y - np.dot(X_LSM, w_LSM[l])
mul = sum([s[i]**2 for i in range(len(s))])
b = mul / (D_size - 2)
b = 1 / b
b # гиперпараметр, найденный методом МНК

5.125224271324821

In [ ]:
a = [x_test[str(i)].tolist() for i in range(x_test.shape[1])]
columns_to_normalize = [str(i) for i in range(x_test.shape[1])]
Xx = pd.DataFrame(make_Furie(pd.DataFrame(np.array(a).T, columns=columns_to_normalize),l+2))
XX = []
for j in range(len(count_safe)):
  XX.insert(j, Xx[count_safe[j]])
XX = pd.DataFrame(np.array(XX).T, columns = count_safe)
s = np.dot(XX, w[l])

In [ ]:
if flag == 3:
  for i in range(x_test.shape[1]): x_test[str(i)] = x_test[str(i)]  * (max_values[i] - min_values[i]) / math.pi
  y_test = [y_test[i] * (max_values[9] - min_values[9]) / math.pi for i in range(len(y_test))]
  s = s * (max_values[9] - min_values[9]) / math.pi
  pd.DataFrame({'y_true':y_test, 'y_found':s})